In [2]:
from tensorflow import keras 
import pandas as pd
import sklearn as sk
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import copy
import numpy as np

In [3]:


def removePlotterAxes():
    plt.figure()
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    pass

def multiPlot(imgs, figsize, no_rows, no_cols):
    plt.figure(figsize = figsize)
    for j in range(len(imgs)):
        plt.subplot(no_rows, no_cols, j+1)
        plt.grid(False)
        plt.xticks([])
        plt.yticks([])
        plt.imshow(imgs[j])

def plotImg(img):
    removePlotterAxes()
    plt.imshow(img)
    pass

def detect(img):
    
    imgToCrop = copy.copy(img)
    #plotImg(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

    faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    eyeCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

    roi = []
    face_imgs = []
    eye_imgs = []
    faces = faceCascade.detectMultiScale(imgToCrop, 1.1, 16)
    for(x, y, w, h) in faces:
        roi.append(imgToCrop[y:y+h, x:x+w])
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
    #plotImg(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

    for j in range(len(roi)):
        face_imgs.append(cv2.cvtColor(roi[j], cv2.COLOR_BGR2RGB))

    eyes = None
    for face in face_imgs:
        #plotImg(face)
        img = face
        imgToCrop = copy.copy(img)
        roi = []
        eyes = eyeCascade.detectMultiScale(face, 1.1, 16)
        for(x, y, w, h) in eyes:
            roi.append(cv2.resize(cv2.cvtColor(imgToCrop[y:y+h, x:x+w], cv2.COLOR_BGR2GRAY), (224,224)))
            cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
        #plotImg(img)
        #if eyes is not None:    
        #    multiPlot(roi, [2, 2], 1, len(eyes))
        
    return np.asanyarray(roi)

In [4]:
model = keras.models.load_model('C:/Users/Moh.Massoud/Desktop/test/DrowsinessDetectionAlexNet.h5')

In [17]:
path = "haarcascade_frontalface_default.xml" 
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + path)
status=""

cap = cv2.VideoCapture(0) # Check if the webcam is opened correctLy 
if not cap.isOpened():
    cap = cv2.VideoCapture('http://192.168.1.4:4747/video') 
if not cap.isOpened(): 
    raise IOError("Cannot open webcam")  

while True:    
     
    ret,frame = cap.read()
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_eye.xml')
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
    #print(faceCoscade.empty()) 
    eyes = eye_cascade.detectMultiScale(gray,1.1,4) 
    for x,y,w,h in eyes:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        eyess = eye_cascade.detectMultiScale(roi_gray) 
        if len(eyess) == 0: 
            pass
            #print("eyes are not detected") 
        else: 
            for (ex,ey,ew,eh) in eyess: 
                eyes_roi = roi_color[ey: ey+eh, ex:ex + ew] 
    final_image = cv2.cvtColor(eyes_roi, cv2.COLOR_BGR2GRAY)                               
    final_image = cv2.resize(final_image, (224,224)) 
    
    final_image = np.expand_dims(final_image,axis =0) ## need fourth dimension 
    final_image = np.expand_dims(final_image,axis =3) ## need fourth dimension 
    #final_image=final_image/255.0 
    eyes_=detect(frame)
    if not eyes_.shape[0]:
        continue
    test_images = np.expand_dims(eyes_, axis=3)
    predictions = model.predict(final_image) 
    Open=True
    for i in range(len(predictions)):
        predict=np.argmax(predictions[i])
        if predict == 0:
            Open=False
    if Open:
        status = 'Open Eyes'
    else:
        status = 'Closed Eyes'

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
    #print(faceCascade.empty())
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30),
        flags=cv2.CASCADE_SCALE_IMAGE
        )
    # Draw o rectangLe around the faces 
    for(x, y, w, h) in faces: 
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2) 
    font = cv2.FONT_HERSHEY_SIMPLEX 
    # Use putText() method for 
    # inserting text on video 
    cv2.putText(frame, status, (50, 50), font, 3, (0, 0, 255), 2, cv2.LINE_4)
    cv2.imshow('Drowsiness Detection',frame) 
    if cv2.waitKey(2) & 0xFF == ord('q'): 
        break 
cap. release() 
cv2.destroyAllWindows() 

KeyboardInterrupt: 

In [16]:
cap. release() 
cv2.destroyAllWindows() 